**Instructions for running this notebook on Colab**
1. Run the cell below to download code from GitHub and install correct version of Python modules
2. Restart session so that Colab can use the newly installed modules rather than their previously installed versions
3. Run all cells

In [1]:
# run this cell to download data and necessary modules
import os, shutil
repo = 'fp-solvers'
if os.path.isdir(repo):
  shutil.rmtree(repo)
!git clone https://github.com/pinakm9/fp-solvers.git
!pip install -r fp-solvers/requirements.txt 2> /dev/null

Cloning into 'fp-solvers'...
remote: Enumerating objects: 15236, done.
remote: Counting objects: 100% (606/606), done.
remote: Compressing objects: 100% (337/337), done.
remote: Total 15236 (delta 262), reused 596 (delta 258), pack-reused 14630
Receiving objects: 100% (15236/15236), 849.97 MiB | 16.97 MiB/s, done.
Resolving deltas: 100% (7417/7417), done.
Updating files: 100% (14754/14754), done.
/


In [ ]:
# add modules folder to Python's search path
import os, sys
from pathlib import Path
script_dir = Path(os.path.dirname(os.path.abspath('')))
module_dir = str(script_dir)
sys.path.insert(0, repo + '/modules')
print(module_dir)

# import the rest of the modules
%matplotlib nbagg
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import arch
import pandas as pd
import tensorflow_probability as tfp
import time
import sim
import utility as ut
import math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
tfd = tfp.distributions

In [2]:
# set up computation parameters
dim = 3
n_particles = int(1e7)
n_subdivs = 100
save_folder = '{}/one-step-filter/L63-pf'.format(repo)
n_steps = 3
n_repeats = 200
dt = 0.01
alpha, beta, rho = 10., 8./3., 28.
t = dt * n_steps
DTYPE = 'float32'
seed = 42
np.random.seed(seed=seed)
tf.random.set_seed(seed=seed)

#### Define $\mu, \sigma, p_0, \sigma_h$

In [3]:
def mu_np(X):
    x, y, z = np.split(X, dim, axis=-1)
    p = alpha * (y - x)
    q = x * (rho - z) - y
    r = x * y - beta * z
    return np.concatenate([p, q, r], axis=-1)

sigma = 10.
sigma_h = 5.

l = np.ones(dim, dtype=DTYPE)
g1 = tfd.MultivariateNormalDiag(loc=2.*l, scale_diag=l)
g2 = tfd.MultivariateNormalDiag(loc=-2.*l, scale_diag=l)
mix = 0.5
rv0 = tfd.Mixture(cat=tfd.Categorical(probs=[mix, 1.-mix]), components=[g1, g2])
log_p0 = lambda x: tf.reshape(rv0.log_prob(x), (-1, 1))

#### Define observations

In [4]:
def H(X):
    x, y, z = np.split(X, 3, axis=-1)
    return np.concatenate([x, z], axis=-1)

def obs(X):
    x, y, z = np.split(X, 3, axis=-1)
    x = x + np.random.normal(scale=sigma_h, size=x.shape).astype(DTYPE)
    z = z + np.random.normal(scale=sigma_h, size=z.shape).astype(DTYPE)
    return np.concatenate([x, z], axis=-1)

#### Set up resampling

In [5]:
def get_weights(y, X):
    rv1 = tfd.MultivariateNormalDiag(loc=y, scale_diag=sigma_h*np.ones(len(y)).astype(DTYPE))
    w = rv1.prob(H(X)).numpy()
    return w/w.sum()

@ut.timer
def systematic_noisy_resample(X, weights, resampling_cov=0.1):
        # make N subdivisions, and choose positions with a consistent random offset
        positions = (np.random.random() + np.arange(n_particles)) / n_particles
        indices = np.zeros(n_particles, 'i')
        cumulative_sum = np.cumsum(weights.astype('float64'))
        i, j = 0, 0

        while i < n_particles:
            if positions[i] < cumulative_sum[j]:
                indices[i] = j
                i += 1
            else:
                j += 1
        indices = list(set(indices))
        offsprings = [0] * len(indices)
        weight_sum = sum([weights[i] for i in indices])
        for k, i in enumerate(indices):
            offsprings[k] = math.ceil(weights[i]/weight_sum*n_particles)
        new_particles = np.zeros((sum(offsprings), dim))
        mean = np.zeros(dim)
        cov = resampling_cov * np.identity(dim)
        j = 0
        for k, i in enumerate(indices):
            new_particles[j] = X[i]
            new_particles[j+1: j+offsprings[k]]= X[i] + np.random.multivariate_normal(mean, cov, size=offsprings[k] - 1)
            j += offsprings[k]
        particles = np.array([new_particles[i] for i in np.random.choice(sum(offsprings), n_particles, replace=False)])
        return particles

#### Load true trajectory and observation

In [6]:
true_state = np.load('{}/true_state.npy'.format(save_folder)).astype(DTYPE)
observation = np.load('{}/observation.npy'.format(save_folder)).astype(DTYPE)

#### Run trajectories

In [7]:
mc = sim.MCProb(save_folder, n_subdivs, mu_np, sigma, rv0.sample(n_particles).numpy())
mc.propagate(n_steps, dt)

Time taken by propagate is 34.1965274810791 seconds


In [ ]:
X = np.genfromtxt('{}/ensemble.csv'.format(mc.save_folder), delimiter=',').astype(DTYPE)
weights = get_weights(observation[0], X)
X_r = systematic_noisy_resample(X, weights, 0.5)
pd.DataFrame(X_r).to_csv('{}/ensemble.csv'.format(mc.save_folder), index=None, header=None)

#### Do box counting

In [ ]:
mc.set_grid(lims=None)
mc.assign_pts()
p_1f = mc.compute_p2(0, 1, save=False)
np.save(save_folder + '/p_1f.npy', p_1f)
p_2f = mc.compute_p2(1, 2, save=False)
np.save(save_folder + '/p_2f.npy', p_2f)
p_3f = mc.compute_p2(2, 0, save=False)
np.save(save_folder + '/p_3f.npy', p_3f)

#### Save grid

In [ ]:
low = mc.grid.mins
high = mc.grid.maxs
x = np.linspace(low[0], high[0], num=n_subdivs+1)[:-1].astype(DTYPE) + mc.grid.h[0]/2.
y = np.linspace(low[1], high[1], num=n_subdivs+1)[:-1].astype(DTYPE) + mc.grid.h[1]/2.
z = np.linspace(low[2], high[2], num=n_subdivs+1)[:-1].astype(DTYPE) + mc.grid.h[2]/2.
np.save('{}/x.npy'.format(save_folder), x)
np.save('{}/y.npy'.format(save_folder), y)
np.save('{}/z.npy'.format(save_folder), z)

In [ ]:
observation[0]